In [13]:
vezzo = open('../vezzo.txt', 'r', encoding='UTF-16').read()
jack = open('../jack.txt', 'r').read()

vezzo = vezzo.replace('\n', ' ').replace('\r', ' ').replace(' ', '')
jack = jack.replace('\n', ' ').replace('\r', ' ').replace(' ', '')
for i in range(0, len(vezzo)):
    if vezzo[i] != jack[i]:
        print(i)
        break
print('All good')

All good


In [13]:
%%time
n = 20 
for i in range(2**n):
            if i%1e6 == 0: print(f'row{i:,}/{2**n:,}')
            vars = [ True if (i & (1 << j)) != 0 else False for j in range(n-1, -1, -1)]

row0/1,048,576
row1,000,000/1,048,576
CPU times: total: 781 ms
Wall time: 2.44 s


In [4]:
%%time
vars = ['x', 'y']
from itertools import product
a = list(product([True, False], repeat=2))
print(a)

[(True, True), (True, False), (False, True), (False, False)]
CPU times: total: 0 ns
Wall time: 0 ns


In [10]:
%%time

def check_valid(expr):
    if ('or' in expr) and ('not' in expr):
            raise Exception(f'Conflicts of operators in <{expr}>')
    if ('and' in expr) and ('not' in expr):
            raise Exception(f'Conflicts of operators in <{expr}>')
    if ('and' in expr) and ('or' in expr):
            raise Exception(f'Conflicts of operators in <{expr}>')
    return True

def cast_list(input_list):
    stack = []
    result = []
    temp = []

    for item in input_list:
        if item == '(':
            check_valid(result)
            stack.append(result)
            result = []
        elif item == ')':
            if stack:
                check_valid(result)
                temp = result
                result = stack.pop()
                result.append(temp)
        else:
            result.append(item)
    
    check_valid(result)
    return result

def evaluate_expression(expr, row, cache):

    def solve(expr):
        # Base case
        if str(expr) in row:
            return row[str(expr)]
        elif str(expr) == 'False':
            return False
        elif str(expr) == 'True':
            return True
        elif str(expr) in cache:
            return cache[str(expr)]
        
        # Short-circuit evaluation
        for i in range(len(expr)):
            if isinstance(expr[i], list):
                if str(expr[i]) in cache:
                    expr[i] = cache[str(expr[i])]
            elif expr[i] in row:
                expr[i] = row[expr[i]]
        
        if ('and' in expr) and (False in expr):
            # print('shortcut')
            return False
        elif ('or' in expr) and (True in expr):
            # print('shortcut')
            return True
        
        
        # Recursion
        while 'not' in expr:
            ix = expr.index('not')
            key = str(['not', expr[ix+1]])
            cache[key] = not solve(expr[ix+1])
            expr[ix:ix+2] = [cache[key]]

        while 'and' in expr:
            ix = expr.index('and')
            key = str([expr[ix-1], 'and', expr[ix+1]])
            a = solve(expr[ix-1])
            if not a:
                cache[key] = False
                expr[ix-1:ix+2] = [cache[key]]
            cache[key] = a and solve(expr[ix+1])
            expr[ix-1:ix+2] = [cache[key]]
        
        while 'or' in expr:
            ix = expr.index('or')
            key = str([expr[ix-1], 'or', expr[ix+1]])
            a = solve(expr[ix-1])
            if a:
                cache[key] = True
                expr[ix-1:ix+2] = [cache[key]]
            cache[key] = a or solve(expr[ix+1]) 
            expr[ix-1:ix+2] = [cache[key]]   

        return expr[0]  
        
    return solve(expr)


expr = ['(', '(', 'not', 'x6', ')', 'or', '(', 'x4', 'or', '(', '(', '(', 'not', 'x1', ')', 'or', 'x2', 'or', 'x7', ')', 'and', '(', 'x0', 'or', '(', 'not', 'x3', ')', 'or', '(', 'not', 'x7', ')', ')', ')', ')', ')', 'and', 'x1', 'and', 'x3', 'and', '(', '(', 'not', 'x2', ')', 'or', '(', 'x7', 'and', '(', '(', 'not', 'x0', ')', 'or', '(', '(', 'not', 'x5', ')', 'and', '(', 'x3', 'or', 'x7', ')', ')', ')', 'and', '(', 'x1', 'or', '(', 'not', 'x3', ')', ')', 'and', '(', 'not', 'x6', ')', 'and', '(', 'x0', 'or', 'x4', 'or', 'x6', ')', ')', ')', 'and', '(', '(', '(', 'not', 'x5', ')', 'and', '(', '(', 'x5', 'or', '(', '(', 'x6', 'or', '(', 'x7', 'or', '(', '(', 'x0', 'or', '(', 'not', 'x1', ')', 'or', 'x3', ')', 'and', '(', 'x1', 'or', 'x2', 'or', '(', 'not', 'x3', ')', ')', ')', ')', ')', 'and', '(', '(', 'not', 'x7', ')', 'or', '(', '(', 'x0', 'or', 'x1', 'or', 'x3', 'or', '(', 'not', 'x6', ')', ')', 'and', '(', '(', 'not', 'x0', ')', 'or', '(', 'not', 'x2', ')', ')', ')', ')', ')', ')', 'and', '(', 'x0', 'or', 'x3', 'or', '(', 'not', 'x5', ')', ')', 'and', 'x1', ')', ')', 'or', '(', 'not', 'x4', ')', ')', 'and', '(', 'not', 'x4', ')', 'and', 'x0', 'and', '(', 'not', 'x2', ')', 'and', '(', 'not', 'x2', ')', 'and', '(', 'not', 'x2', ')', 'and', 'x6', 'and', '(', 'not', 'x2', ')', 'and', 'x7']
expr = cast_list(expr)
print(expr)
vars=['x7', 'x6', 'x5', 'x4', 'x3', 'x2', 'x1', 'x0']
n = len(vars)
for i in range(2**n):
    var = dict(zip(vars, [ True if (i & (1 << j)) != 0 else False for j in range(n-1, -1, -1)]))
    print(var)       
    cache = {}
    out = evaluate_expression(expr.copy(), var, cache)
    print(out)

[[['not', 'x6'], 'or', ['x4', 'or', [[['not', 'x1'], 'or', 'x2', 'or', 'x7'], 'and', ['x0', 'or', ['not', 'x3'], 'or', ['not', 'x7']]]]], 'and', 'x1', 'and', 'x3', 'and', [['not', 'x2'], 'or', ['x7', 'and', [['not', 'x0'], 'or', [['not', 'x5'], 'and', ['x3', 'or', 'x7']]], 'and', ['x1', 'or', ['not', 'x3']], 'and', ['not', 'x6'], 'and', ['x0', 'or', 'x4', 'or', 'x6']]], 'and', [[['not', 'x5'], 'and', [['x5', 'or', [['x6', 'or', ['x7', 'or', [['x0', 'or', ['not', 'x1'], 'or', 'x3'], 'and', ['x1', 'or', 'x2', 'or', ['not', 'x3']]]]], 'and', [['not', 'x7'], 'or', [['x0', 'or', 'x1', 'or', 'x3', 'or', ['not', 'x6']], 'and', [['not', 'x0'], 'or', ['not', 'x2']]]]]], 'and', ['x0', 'or', 'x3', 'or', ['not', 'x5']], 'and', 'x1']], 'or', ['not', 'x4']], 'and', ['not', 'x4'], 'and', 'x0', 'and', ['not', 'x2'], 'and', ['not', 'x2'], 'and', ['not', 'x2'], 'and', 'x6', 'and', ['not', 'x2'], 'and', 'x7']
{'x7': False, 'x6': False, 'x5': False, 'x4': False, 'x3': False, 'x2': False, 'x1': False, 'x0'

In [29]:
vars = ['x', 'y', 'z', 'w']
expr = [['x', 'and', 'y'], 'or', ['not', 'z'], 'or' 'w']
expr = ['x', 'and', 'y', 'and', 'z']
class Node:
    def __init__(self, value=None, children=[]):
        self.value = value
        self.children = children
    
    def eval(self, variables):
        
        if self.value == 'and':
            out = self.children[0].eval(variables)
            for child in self.children:
                out = out and child.eval(variables)
                if out == False:
                    return False
            return out
        
        elif self.value == 'or':
            out = self.children[0].eval(variables)
            for child in self.children:
                out = out or child.eval(variables)
                if out == True:
                    return True
            return out
        
        elif self.value == 'not':
            return not self.children[0]
        else:
            return variables[self.value]

    def __repr__(self) -> str:
        return f'<Node> \'{self.value}\' with {len(self.children)} children'
# Function to parse the boolean expression and build a tree
def build_tree(expr):
    
    if isinstance(expr, list):
        node = Node()
        for i in range(len(expr)):
            if expr[i] in ('not', 'and', 'or'):
                node.value = expr[i]
            else:
                node.children.append(build_tree(expr[i]))
        return node
    else:
        return Node(expr)

expr = [['x', 'and', 'y'], 'and', ['not', 'z'], 'and', 'w']
expr = ['(', '(', 'not', 'x6', ')', 'or', '(', 'x4', 'or', '(', '(', '(', 'not', 'x1', ')', 'or', 'x2', 'or', 'x7', ')', 'and', '(', 'x0', 'or', '(', 'not', 'x3', ')', 'or', '(', 'not', 'x7', ')', ')', ')', ')', ')', 'and', 'x1', 'and', 'x3', 'and', '(', '(', 'not', 'x2', ')', 'or', '(', 'x7', 'and', '(', '(', 'not', 'x0', ')', 'or', '(', '(', 'not', 'x5', ')', 'and', '(', 'x3', 'or', 'x7', ')', ')', ')', 'and', '(', 'x1', 'or', '(', 'not', 'x3', ')', ')', 'and', '(', 'not', 'x6', ')', 'and', '(', 'x0', 'or', 'x4', 'or', 'x6', ')', ')', ')', 'and', '(', '(', '(', 'not', 'x5', ')', 'and', '(', '(', 'x5', 'or', '(', '(', 'x6', 'or', '(', 'x7', 'or', '(', '(', 'x0', 'or', '(', 'not', 'x1', ')', 'or', 'x3', ')', 'and', '(', 'x1', 'or', 'x2', 'or', '(', 'not', 'x3', ')', ')', ')', ')', ')', 'and', '(', '(', 'not', 'x7', ')', 'or', '(', '(', 'x0', 'or', 'x1', 'or', 'x3', 'or', '(', 'not', 'x6', ')', ')', 'and', '(', '(', 'not', 'x0', ')', 'or', '(', 'not', 'x2', ')', ')', ')', ')', ')', ')', 'and', '(', 'x0', 'or', 'x3', 'or', '(', 'not', 'x5', ')', ')', 'and', 'x1', ')', ')', 'or', '(', 'not', 'x4', ')', ')', 'and', '(', 'not', 'x4', ')', 'and', 'x0', 'and', '(', 'not', 'x2', ')', 'and', '(', 'not', 'x2', ')', 'and', '(', 'not', 'x2', ')', 'and', 'x6', 'and', '(', 'not', 'x2', ')', 'and', 'x7']
tree = build_tree(expr)

variables = {
    'x': True,
    'y': True,
    'z': False,
    'w': False
}

print(tree)
result = tree.eval(variables)
print(f"Result of the expression: {result}")

<Node> 'and' with 3 children
Result of the expression: False


In [3]:
class Node:
    def __init__(self, value=None, children=[]):
        self.value = value
        self.children = children.copy()
    
    def eval(self, variables):
        
        if self.value == 'and':
            out = self.children[0].eval(variables)
            for child in self.children:
                out = out and child.eval(variables)
                if out == False:
                    return False
            return out
        
        elif self.value == 'or':
            out = self.children[0].eval(variables)
            for child in self.children:
                out = out or child.eval(variables)
                if out == True:
                    return True
            return out
        
        elif self.value == 'not':
            return not self.children[0]
        else:
            return variables[self.value]

    def __repr__(self) -> str:
        return f'<Node> \'{self.value}\' with {len(self.children)} children'

def build_tree_recursively(expr):
    
    def build_tree(expr):
        children = []
        node_type = None
        i = 0
        while i < len(expr):
            token = expr[i]
            
            if token == '(':
                # Find the matching closing parenthesis
                open_parens = 1
                for j in range(i + 1, len(expr)):
                    if expr[j] == '(':
                        open_parens += 1
                    elif expr[j] == ')':
                        open_parens -= 1
                    if open_parens == 0:
                        # Solve the sub-expr within the parentheses
                        node = build_tree(expr[i + 1:j])
                        children.append(node)
                        i = j  # Move the index to after the closing ')'
                        break
            
            elif token in ('not', 'and', 'or'):
                node_type = token if node_type is None else node_type
            else:
                children.append(Node(token))

            i += 1
        node = Node(node_type, children)
        return node
    
    return build_tree(expr)

expr = ['(', 'x', 'and', 'y', ')', 'and', '(', 'not', 'z', ')', 'and', 'w']
expr = ['(', '(', 'not', 'x6', ')', 'or', '(', 'x4', 'or', '(', '(', '(', 'not', 'x1', ')', 'or', 'x2', 'or', 'x7', ')', 'and', '(', 'x0', 'or', '(', 'not', 'x3', ')', 'or', '(', 'not', 'x7', ')', ')', ')', ')', ')', 'and', 'x1', 'and', 'x3', 'and', '(', '(', 'not', 'x2', ')', 'or', '(', 'x7', 'and', '(', '(', 'not', 'x0', ')', 'or', '(', '(', 'not', 'x5', ')', 'and', '(', 'x3', 'or', 'x7', ')', ')', ')', 'and', '(', 'x1', 'or', '(', 'not', 'x3', ')', ')', 'and', '(', 'not', 'x6', ')', 'and', '(', 'x0', 'or', 'x4', 'or', 'x6', ')', ')', ')', 'and', '(', '(', '(', 'not', 'x5', ')', 'and', '(', '(', 'x5', 'or', '(', '(', 'x6', 'or', '(', 'x7', 'or', '(', '(', 'x0', 'or', '(', 'not', 'x1', ')', 'or', 'x3', ')', 'and', '(', 'x1', 'or', 'x2', 'or', '(', 'not', 'x3', ')', ')', ')', ')', ')', 'and', '(', '(', 'not', 'x7', ')', 'or', '(', '(', 'x0', 'or', 'x1', 'or', 'x3', 'or', '(', 'not', 'x6', ')', ')', 'and', '(', '(', 'not', 'x0', ')', 'or', '(', 'not', 'x2', ')', ')', ')', ')', ')', ')', 'and', '(', 'x0', 'or', 'x3', 'or', '(', 'not', 'x5', ')', ')', 'and', 'x1', ')', ')', 'or', '(', 'not', 'x4', ')', ')', 'and', '(', 'not', 'x4', ')', 'and', 'x0', 'and', '(', 'not', 'x2', ')', 'and', '(', 'not', 'x2', ')', 'and', '(', 'not', 'x2', ')', 'and', 'x6', 'and', '(', 'not', 'x2', ')', 'and', 'x7']
expr = ['x', 'and', '(', 'not', 'z', ')', 'and', 'w']
root = build_tree_recursively(expr)
root.children

[<Node> 'x' with 0 children,
 <Node> 'not' with 1 children,
 <Node> 'w' with 0 children]